В этом блоке проанализируем А/Б тест

Мы добавили на лендинг с оформлением дебетовой карты (https://www.tinkoff.ru/cards/debit-cards/tinkoff-black/) информацию о дополнительных кэшбеках, которые появляются благодаря нашим патнерам. Рассчитываем, что это вырастит конверсию из переходов на лендинг в оформление дебетовой карты. 

В приложенном файле вам доступны логи переходов на сайт и последующего оформления карточки за каждый день. 

In [1]:
# загрузим необходимые пакеты

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats


In [2]:
# Загрузите таблицу в датафрейм

df = pd.read_csv('hw_ab.csv')

In [3]:
# Посмотрите на поля и проанализируйте таблицу: она содержит поля 
# date — дата захода пользователя на лендинг
# id — идентификатор, присвоенный пользователю
# group — группа теста или контроля
# converted – 1, если пользователь оформил карточку и 0, если не оформил



In [4]:
df

,date,id,group,converted
0,2023-08-01,5030836,control,0
1,2023-08-01,5091425,control,0
2,2023-08-01,5106537,control,0
3,2023-08-01,4556522,control,0
4,2023-08-01,4002917,control,0
...,...,...,...,...
94773,2023-11-01,7729860,test,0
94774,2023-11-01,8088716,test,0
94775,2023-11-01,4478281,test,0
94776,2023-11-01,5710762,test,0


In [5]:
df.info(),df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94778 entries, 0 to 94777
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       94778 non-null  object
 1   id         94778 non-null  int64 
 2   group      94778 non-null  object
 3   converted  94778 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.9+ MB


(None,
          date       id    group  converted
 0  2023-08-01  5030836  control          0
 1  2023-08-01  5091425  control          0
 2  2023-08-01  5106537  control          0
 3  2023-08-01  4556522  control          0
 4  2023-08-01  4002917  control          0)

In [6]:
# убедитесь,что пользователи уникальные


In [7]:
unique_users = df['id'].nunique()
unique_users

94778

In [8]:
# убедитесь, что пользователей в тесте и контроле приблизительно равное количество, 
# в каждой группе каждый день теста были пользователи, и они как оформляли карту, так и нет



In [9]:
group_couts = df['group'].value_counts()
group_couts

group
test       47448
control    47330
Name: count, dtype: int64

In [10]:
valid_days = df.groupby(['date', 'group'])['converted'].agg(
    has_conv = lambda  x:(x==1).any(),
    has_not_conv = lambda x: (x==0).any()
).reset_index()

valid_days

,date,group,has_conv,has_not_conv
0,2023-08-01,control,True,True
1,2023-08-01,test,True,True
2,2023-08-02,control,True,True
3,2023-08-02,test,True,True
4,2023-08-03,control,True,True
...,...,...,...,...
181,2023-10-30,test,True,True
182,2023-10-31,control,True,True
183,2023-10-31,test,True,True
184,2023-11-01,control,True,True


In [11]:
# Флаги наличия хотябы одного конвентирванного и неконв
invalid_days = valid_days[~(valid_days['has_conv']&valid_days['has_not_conv'])]
invalid_days

,date,group,has_conv,has_not_conv


In [12]:
# отличаются ли конверсии в тесте и в контроле?



In [13]:
# конверсия в контроле: 
control = df[df['group']=='control'].mean()
conversion_control = control


In [14]:
# конверсия в тесте: 


In [15]:
# достаточно ли длинным был тест, чтобы мы могли посчитать средние конверсии и сравнить значимость их различия? 
# посчитайте количество дней теста, убедитесь, что среди них нет пропусков

